## 1. Aave position

- Retrieve by account (address) and asset (WETH)
  - Current collateral
  - Borrowed value
  - Health

## 2. Uniswap Position

- Retrieve all position by account (address)
  - Current position (after impermanent loss)
  - Reward

## 3. Daily valuation

- Retrieve up-to-date prices for tokens


In [1]:
from credmark.cmf.ipython import create_cmf
from credmark.cmf.types import (Address, BlockNumber, Contract, Token, Portfolio, PriceWithQuote)
from credmark.cmf.types.data.fiat_currency_data import FIAT_CURRENCY_DATA_BY_SYMBOL
from credmark.cmf.model.errors import ModelRunError, ModelDataError
from web3 import HTTPProvider, Web3
from datetime import datetime, timezone, timedelta
from models.credmark.tokens.token import get_eip1967_proxy, get_eip1967_proxy_err

from models.tmp_abi_lookup import AAVE_STABLEDEBT_ABI, AAVE_LENDING_POOL_PROVIDER, AAVE_DATA_PROVIDER, AAVE_LENDING_POOL_PROXY, AAVE_LENDING_POOL

from models.credmark.protocols.lending.aave.aave_v2_util import get_aave_lender
from models.credmark.protocols.lending.aave.aave_v2_account import AAVEUserReserveData

cmf_param = {
    'block_number': 40822399,
    # 'chain_to_provider_url': {'137': 'https://polygon-rpc.com', '56': 'https://bsc-dataseed.binance.org'},
    'api_url': None,
    'use_local_models': '*',  # use local to speed up
    'chain_id': 137,
}

# When using fill_db_evt in a loop, create cmf context outside the loop and sent _cmf_param = None
# When using fill_db_evt standalone, either 1) create cmf context outside the loop and sent _cmf_param = None, 2) sent in non-None cmf_param.
context, _model_loader = create_cmf(cmf_param)
context._web3.eth.default_block = int(context.block_number)
end_block = int(context.block_number)

In [2]:
usdc = Token('USDC')
weth = Token('WETH')
dpi = Token('DPI')
sushi = Token('SUSHI')
wmatic = Token('WMATIC')
link = Token('LINK')
ghst = Token('GHST')
sushi = Token('SUSHI')

for t in [usdc, weth, sushi, wmatic, link, ghst, sushi, dpi]:
    print(t)
    print(context.run_model('price.oracle-chainlink', {'base': t}))

address='0x2791bca1f2de4661ed88a30c99a7a9449aa84174'
{'price': 0.99965426, 'src': 'chainlink.price-by-feed|USDC / USD|0xf9c53A834F60cBbE40E27702276fBc0819B3aFAD|v4|None|t:0s|r:0', 'quoteAddress': '0x0000000000000000000000000000000000000348'}
address='0x7ceb23fd6bc0add59e62ac25578270cff1b9f619'
{'price': 1767.84839107, 'src': 'chainlink.price-by-feed|ETH / USD|0x4dD6655Ad5ed7C06c882f496E3f42acE5766cb89|v4|None|t:8s|r:0', 'quoteAddress': '0x0000000000000000000000000000000000000348'}
address='0x0b3f868e0be5597d5db7feb59e1cadbb0fdda50a'
{'price': 1.0385, 'src': 'chainlink.price-by-feed|SUSHI / USD|0xf3F28072f7FF2510843504c69ACD07C2C84B2d83|v4|None|t:288s|r:0', 'quoteAddress': '0x0000000000000000000000000000000000000348'}
address='0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270'
{'price': 1.083, 'src': 'chainlink.price-by-feed|MATIC / USD|0x1278C74c3B2f8c3BcA0089b4E128fAf023615ECf|v4|None|t:4s|r:0', 'quoteAddress': '0x0000000000000000000000000000000000000348'}
address='0x53e0bca35ec356bd5dddfebb

In [3]:
from credmark.cmf.model import Model
from credmark.cmf.model.errors import ModelDataError, ModelRunError


@Model.describe(
    slug='jit.aave-user-reserve',
    version='1.0',
    input=dict,
    output=dict)
class JitAAVEUserReserve(Model):
    def run(self, input):
        data_provider = self.context.run_model('aave-v2.get-protocol-data-provider', {}, return_type=Contract)
        data_provider.set_abi(AAVE_DATA_PROVIDER, set_loaded=True)

        return AAVEUserReserveData(*data_provider.functions.getUserReserveData(
            Address(input['asset']).checksum,
            Address(input['account']).checksum).call()).normalize()._asdict()


context.add_model(JitAAVEUserReserve)

In [4]:
account = '0x1d8b86e3D88cDb2d34688e87E72F388Cb541B7C8'
account = '0xea9CC42d89C4f04E1583905B7De5a9f5F292Daa5'
account = '0xd2A94C6e3b8A5B479AeD45Fd7D2726890dE325E5'

# On earlier block: 40782399

prev = context.run_model('jit.aave-user-reserve',
                         input={'asset': weth.address, 'account': account}, block_number=40782399)
now = context.run_model('jit.aave-user-reserve', input={'asset': weth.address, 'account': account})

(prev, now)

({'currentATokenBalance': 2.9953183172440756,
  'currentStableDebt': 0.0,
  'currentVariableDebt': 0.0,
  'principalStableDebt': 0.0,
  'scaledVariableDebt': 0.0,
  'stableBorrowRate': 0.0,
  'liquidityRate': 0.0007052660597053956,
  'stableRateLastUpdated': 0,
  'usageAsCollateralEnabled': True},
 {'currentATokenBalance': 3.0161404016426845,
  'currentStableDebt': 0.0,
  'currentVariableDebt': 0.0,
  'principalStableDebt': 0.0,
  'scaledVariableDebt': 0.0,
  'stableBorrowRate': 0.0,
  'liquidityRate': 0.0007098184012045643,
  'stableRateLastUpdated': 0,
  'usageAsCollateralEnabled': True})

In [5]:
prev = context.run_model('uniswap-v3.id', {'id': 799208}, block_number=40782399)

now = context.run_model('uniswap-v3.id', {'id': 799208})

prev, now

({'lp': '0x3aafea8e51999a80d3ce76e27b7cabbe2a48258c',
  'id': 799208,
  'pool': '0x4fe1269a585b141f11c3e144158f9f8823c7c0e7',
  'tokens': [{'amount': 0.010980546729303646,
    'asset': {'address': '0x7ceb23fd6bc0add59e62ac25578270cff1b9f619'},
    'price_quote': None,
    'fee': 0.0004967996056704618},
   {'amount': 82.03222412663655,
    'asset': {'address': '0x9a71012b13ca4d3d0cdc72a177df3ef03b0e76a3'},
    'price_quote': None,
    'fee': 0.12175070605035752}],
  'in_range': 'in range'},
 {'lp': '0x3aafea8e51999a80d3ce76e27b7cabbe2a48258c',
  'id': 799208,
  'pool': '0x4fe1269a585b141f11c3e144158f9f8823c7c0e7',
  'tokens': [{'amount': 0.01320629553518665,
    'asset': {'address': '0x7ceb23fd6bc0add59e62ac25578270cff1b9f619'},
    'price_quote': None,
    'fee': 0.0006672336099092304},
   {'amount': 81.44496589714221,
    'asset': {'address': '0x9a71012b13ca4d3d0cdc72a177df3ef03b0e76a3'},
    'price_quote': None,
    'fee': 0.16452881831416938}],
  'in_range': 'in range'})

In [6]:
lp_pos = context.run_model('uniswap-v3.lp', {'lp': '0x470cB7e9981Db525422A16A21d8cD510B0766d17'})
len(lp_pos['positions'])

89

In [7]:
weth.address

Token('WETH').address

'0x7ceb23fd6bc0add59e62ac25578270cff1b9f619'

In [8]:
prev = context.run_model('price.oracle-chainlink',
                         {'base': '0x9a71012b13ca4d3d0cdc72a177df3ef03b0e76a3'}, block_number=40782399)

now = context.run_model('price.oracle-chainlink', {'base': '0x9a71012b13ca4d3d0cdc72a177df3ef03b0e76a3'})

(prev, now)

({'price': 6.63236991,
  'src': 'chainlink.price-by-feed|BAL / USD|0xd0CCf213410578DF4EC5EB0157234120B30d2f81|v4|None|t:9315s|r:0',
  'quoteAddress': '0x0000000000000000000000000000000000000348'},
 {'price': 6.69169367,
  'src': 'chainlink.price-by-feed|BAL / USD|0xd0CCf213410578DF4EC5EB0157234120B30d2f81|v4|None|t:4266s|r:0',
  'quoteAddress': '0x0000000000000000000000000000000000000348'})